# Install packages

In [46]:
###
###
# install requirements
###
###

# Import packages

In [47]:
import numpy as np
import pandas as pd
import re

from sklearn.base import BaseEstimator, TransformerMixin

from hazm import Normalizer, Stemmer, Lemmatizer, WordTokenizer

# Data loading

In [36]:
tehran_posts = pd.read_csv('data/tehran-apartment-sell-posts.csv')
mashhad_posts = pd.read_csv('data/mashhad-apartment-sell-posts.csv')
isfahan_posts = pd.read_csv('data/isfahan-apartment-sell-posts.csv')

In [37]:
posts = pd.concat([tehran_posts, mashhad_posts, isfahan_posts], ignore_index=True)

# Clean data

In [38]:
posts.head()

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price,Description
0,63,1375,1,False,False,True,دقایقی پیش در تهران، شمشیری، خیابان شمشیری (ب....,۱٬۶۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
1,180,1393,3,True,True,True,دقایقی پیش در تهران، شیان | فروش آپارتمان,۱۴٬۰۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
2,38,1380,1,False,False,True,دقایقی پیش در تهران، پلیس | فروش آپارتمان,۱٬۳۸۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
3,88,1384,2,True,True,True,دقایقی پیش در تهران، تهران‌نو | فروش آپارتمان,۳٬۵۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
4,40,1393,1,True,True,True,دقایقی پیش در تهران، بریانک | فروش آپارتمان,۱٬۴۲۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."


In [39]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Area          1359 non-null   int64 
 1   Construction  1359 non-null   int64 
 2   Room          1359 non-null   int64 
 3   Warehouse     1359 non-null   bool  
 4   Parking       1359 non-null   bool  
 5   Elevator      1359 non-null   bool  
 6   Address       1359 non-null   object
 7   Price         1359 non-null   object
 8   Description   1359 non-null   object
dtypes: bool(3), int64(3), object(3)
memory usage: 67.8+ KB


### Clean Address feature

In [40]:
posts.Address[0]

'دقایقی پیش در تهران، شمشیری، خیابان شمشیری (ب. ایت الله سعیدی) | فروش آپارتمان'

In [42]:
class extractAddressTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X.Address = X.Address.apply(
            lambda adress: '،'.join(adress.split('|')[0].split('،')[1:])
        )
        return X

In [30]:
t = extractAddressTransformer()

In [49]:
t.fit_transform(posts)

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price,Description
0,63,1375,1,False,False,True,شمشیری، خیابان شمشیری (ب. ایت الله سعیدی),۱٬۶۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
1,180,1393,3,True,True,True,شیان,۱۴٬۰۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
2,38,1380,1,False,False,True,پلیس,۱٬۳۸۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
3,88,1384,2,True,True,True,تهران‌نو,۳٬۵۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
4,40,1393,1,True,True,True,بریانک,۱٬۴۲۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
...,...,...,...,...,...,...,...,...,...
1354,87,1396,2,True,True,True,بهارستان,۱٬۰۸۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
1355,130,1401,3,True,True,True,آفاران,۲٬۸۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
1356,122,1393,2,True,True,True,اشراق,۲٬۷۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."
1357,105,1389,2,True,True,True,خانه اصفهان,۲٬۷۰۰٬۰۰۰٬۰۰۰ تومان,"<p class=""kt-description-row__text kt-descript..."


In [58]:
posts.Address[0]

' شمشیری، خیابان شمشیری (ب. ایت الله سعیدی) '

### Clean Description feature

In [43]:
posts.Description[1]

'<p class="kt-description-row__text kt-description-row__text--primary">برج باغ با موقعیت دنج و آرام/ چشم انداز پارک جنگلی و قله دماوند/ 3خواب، یک مستر/ لابی بزرگ/ سرایدار مقیم/ بالکن مفید/ ژنراتور برق/ چیلر مرکزی/ اطفای حریق/دوربین مداربسته/ بدون دیوار مشترک/ نورگیر از شمال، شرق و غرب واحد/ حیاط بزرگ و مشجر با آبنما، آلاچیق و چاه آب/ روف گاردن/ پیاده تا پارک جنگلی ده دقیقه.\nلطفا املاک تماس نگیرند.</p>'

In [53]:
class removeTagTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.cleanre = re.compile('<.*?>') 
        return self
    def transform(self, X):
        X.Description = X.Description.apply(
            lambda desc: re.sub(self.cleanre, '', desc)
        )
        return X

In [54]:
t = removeTagTransformer()
t.fit_transform(posts)

,Area,Construction,Room,Warehouse,Parking,Elevator,Address,Price,Description
0,63,1375,1,False,False,True,شمشیری، خیابان شمشیری (ب. ایت الله سعیدی),۱٬۶۰۰٬۰۰۰٬۰۰۰ تومان,✴️دونبش\n✴️ تک واحدی \n✴️ از سه جهت غرق در نور...
1,180,1393,3,True,True,True,شیان,۱۴٬۰۰۰٬۰۰۰٬۰۰۰ تومان,برج باغ با موقعیت دنج و آرام/ چشم انداز پارک ج...
2,38,1380,1,False,False,True,پلیس,۱٬۳۸۰٬۰۰۰٬۰۰۰ تومان,اپارتمان ۳۸ متری پلیس-کلیم کاشانی \nتکواحدی \n...
3,88,1384,2,True,True,True,تهران‌نو,۳٬۵۰۰٬۰۰۰٬۰۰۰ تومان,مشاورین املاک یونسی\nآدرس املاک : فلکه چایچی خ...
4,40,1393,1,True,True,True,بریانک,۱٬۴۲۰٬۰۰۰٬۰۰۰ تومان,هوالرزاق\n\nبربانک\nابراهیم حسینی\n\n۴۰ متر\nت...
...,...,...,...,...,...,...,...,...,...
1354,87,1396,2,True,True,True,بهارستان,۱٬۰۸۰٬۰۰۰٬۰۰۰ تومان,سلام \nاپارتمان ۸۷ متری دو خوابه مسکن مهر \nجن...
1355,130,1401,3,True,True,True,آفاران,۲٬۸۰۰٬۰۰۰٬۰۰۰ تومان,⭕املاک آسمان آبی⭕\n\n\n⭕مشاور شما عزیزان برای ...
1356,122,1393,2,True,True,True,اشراق,۲٬۷۰۰٬۰۰۰٬۰۰۰ تومان,آپارتمان بسیار خوش جا و کم واحد با نور بسیار ع...
1357,105,1389,2,True,True,True,خانه اصفهان,۲٬۷۰۰٬۰۰۰٬۰۰۰ تومان,واقع در بهترین لوکیشن خانه اصفهان\nدسترسی عالی...


In [57]:
posts.Description[1]

'برج باغ با موقعیت دنج و آرام/ چشم انداز پارک جنگلی و قله دماوند/ 3خواب، یک مستر/ لابی بزرگ/ سرایدار مقیم/ بالکن مفید/ ژنراتور برق/ چیلر مرکزی/ اطفای حریق/دوربین مداربسته/ بدون دیوار مشترک/ نورگیر از شمال، شرق و غرب واحد/ حیاط بزرگ و مشجر با آبنما، آلاچیق و چاه آب/ روف گاردن/ پیاده تا پارک جنگلی ده دقیقه.\nلطفا املاک تماس نگیرند.'